# Paso #1: IMPORTAR LIBRERÍAS

In [ ]:
!wget https://www.dropbox.com/s/i4n134sqzu7psry/fotos%20procesadas.zip?dl=0

In [ ]:
!wget https://www.dropbox.com/s/u5fu6o8285ebsne/Fotos%20TFM.zip?dl=0

In [ ]:
!unzip Fotos\ TFM.zip\?dl\=0

In [ ]:
!unzip fotos\ procesadas.zip\?dl\=0

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Funciones y metodos reutilizables
def elem2dict(node):
    """
    Convert an xml.ElementTree node tree into a dict.
    """
    result = {}

    for element in node:
        key = element.tag
        if '}' in key:
            # Remove namespace prefix
            key = key.split('}')[1]

        if node.attrib:
            result['@attribs'] = dict(node.items())

        # Process element as tree element if the inner XML contains non-whitespace content
        if element.text and element.text.strip():
            value = element.text
        else:
            value = elem2dict(element)

        # Check if a node with this name at this depth was already found
        if key in result:
            if type(result[key]) is not list:
                # We've seen it before, but only once, we need to convert it to a list
                tempvalue = result[key].copy()
                result[key] = [tempvalue, value]
            else:
                # We've seen it at least once, it's already a list, just append the node's inner XML
                result[key].append(value)
        else:
            # First time we've seen it
            result[key] = value

    return result

def get_data_imgLabel(imgLabel):
    x = imgLabel["bndbox"]["xmin"]
    y = imgLabel["bndbox"]["ymin"]
    width = imgLabel["bndbox"]["xmax"]
    height = imgLabel["bndbox"]["ymax"]

    return x,y, width, height

# Funcion para redimensionar la imagen  a (512, 512)
# para redimensionar la imagen  a (512, 512)
def resize(x):
  return cv2.resize(x, dsize=(widthImage, heightImage), interpolation = cv2.INTER_AREA)

In [ ]:
# Importamos los paquetes necesarios
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json
from xml.etree import ElementTree as ET
import random
from PIL import Image

# Definimos variables generales
# Ruta de la data y capertas dentro de esta
dataPath = '/content/Fotos TFM'
dataPathXML = '/content/fotos procesadas'
typeLeavesList = os.listdir(dataPathXML)

widthImage = 224
heightImage = 224
quantitytoProcess = 1000

# data
labels = []
leavesData = []
labelCounter = 0

# Leemos la información de los Json en las carpetas, para 
# obtener las coordenadas de las imagenes etiquetadas.
listCount = []
dataToProcess = []

for nameDir in typeLeavesList:
    leavePathXML = dataPathXML + '/' + nameDir
    leavePathImg = dataPath + '/' + nameDir

    if "Sanas" == nameDir:
      labelCounter = 0

    if "Enfermas" == nameDir:
      labelCounter = 1

    # lista de archivos .json con las etiquetas de las imagenes
    listXML =  [f for f in os.listdir(leavePathXML) if f.endswith('.xml')]

    listXML = listXML[:quantitytoProcess]

    countFilesXML = len([name for name in listXML ])
    print('Cantidad de imágenes:', nameDir)
    print(countFilesXML)

    sample20ListXML = random.sample(listXML, 20)

    # Cantidad de archivos a procesar
    listCount.append(countFilesXML)

    for fileXML in listXML:
        listFiltered = list(filter(lambda x: x["label"] == labelCounter, dataToProcess))
        if len(listFiltered) > quantitytoProcess:
            continue

        xmlParsed = ET.parse(leavePathXML + '/'+ fileXML).getroot()
        xmlDict = elem2dict(xmlParsed)
        imageName = xmlDict["filename"]

        if "object" not in xmlDict:
            continue

        objectImgLabel = xmlDict["object"]

        # Obtenemos la imagen completa
        image = cv2.imread(leavePathImg+'/'+ imageName)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if (type(objectImgLabel) == list):
            for imgLabel in objectImgLabel:
                x, y, width, height = get_data_imgLabel(imgLabel)
                roi = image[int(y):int(height), int(x):int(width)]

                leafData = {
                    "filenameImg": imageName,
                    "filenameXML": fileXML,
                    "label" : labelCounter,
                    "roi": resize(roi)
                }
                dataToProcess.append(leafData)

        else:
            x, y, width, height = get_data_imgLabel(objectImgLabel)
            roi = image[int(y):int(height), int(x):int(width)]
            leafData = {
                "filenameImg": imageName,
                "filenameXML": fileXML,
                "label" : labelCounter,
                "roi": resize(roi)
            }
            dataToProcess.append(leafData)


newListEnfermas = filter(lambda x: x["label"] == 1, dataToProcess)
newListSanas = filter(lambda x: x["label"] == 0, dataToProcess)

listEnfermas = list(newListEnfermas)
listSanas = list(newListSanas)


In [ ]:
# # Mostrar las primeras 40 imágenes enfermas
fig = plt.figure(figsize=(15, 15))

for i in range(40):
    image = listEnfermas[i]["roi"]
    ax = fig.add_subplot(5, 8, i + 1)
    ax.imshow(image)
    ax.axis("off")

plt.show()

In [ ]:
# # Mostrar las primeras 40 imágenes SANAS
fig = plt.figure(figsize=(15, 15))

for i in range(40):
    image = listSanas[i]["roi"]
    ax = fig.add_subplot(5, 8, i + 1)
    ax.imshow(image)
    ax.axis("off")

plt.show()

In [ ]:
# Visualiacion de cantidad de imagenes por clasificación - Enfermas / Sanas
xbarPlot = ["Enfermas", "Sanas"]
ybarPlot = [len(listEnfermas), len(listSanas)]

plt.figure(figsize = (5,5))
sns.barplot(x = xbarPlot, y = ybarPlot )

In [ ]:
print('Número de etiquetas 1 (Enfermas): ', len(listEnfermas))
print('Número de etiquetas 0(Sanas): ', len(listSanas))

# Paso #2: VISUALIZACIÓN DE IMÁGENES

# Paso #3: PREPROCESADOR DE LAS IMÁGENES

In [ ]:
# Dividir el dataframe en características y etiquetas para la clasificación
from keras.utils import to_categorical

X = np.array([x["roi"] for x in dataToProcess])
y = to_categorical([x["label"] for x in dataToProcess] )

# # Visualizamos los pixeles de las imagenes
X[0]

In [ ]:
# Visualizamos las etiquetas
y

In [ ]:
# Revisando la dimensionalidad de X
X.shape

In [ ]:
X = np.stack(X, axis = 0)
# aqui era uno, pero algo ha pasado y se cambio a 3
X = X.reshape(len(listEnfermas)+len(listSanas) , 224, 224, 3)

print(X.shape, y.shape)

In [ ]:
# Dividir el dataframe en conjunto de entrenamiento, test y validación

from sklearn.model_selection import train_test_split

# Obtenemos el conjunto de entrenamiento y el conjunto de Test
X_train, X_Test, y_train, y_Test = train_test_split(X, y, test_size = 0.4, shuffle = True)

# Del conjunto de Test, obtenemos una parte para el conjunto de validaciòn, el 50% del conjunto de Test
X_val, X_Test, y_val, y_Test = train_test_split(X_Test, y_Test, test_size = 0.6, shuffle = True)

In [ ]:
# Limpiar memoria
del dataToProcess
del X
del y


In [ ]:
print("Conjunto de validación")
print(X_val.shape, y_val.shape)
print("Conjunto de Test")
print(X_Test.shape, y_Test.shape)
print("Conjunto de Entrenamiento")
print(X_train.shape, y_train.shape)

# Paso #4: NORMALIZACIÓN DE LOS DATOS Y PREPARACIÓN PARA EL ENTRENAMIENTO

In [ ]:
# # Pre Procesado de Imágenes
# with tf.device('/gpu:0'):
#   X_train = X_train/255
#   X_val   = X_val /255
#   X_Test  = X_Test/255

#   X_train

# Paso #3: AUMENTACIÓN DE LAS IMÁGENES

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=90,
    # width_shift_range=0.05,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # vertical_flip = True,
    brightness_range = [1.1, 1.15]
    )

valid_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # vertical_flip = True,
    brightness_range = [1.1, 1.15]
    )


# TAREA #6: CONSTRUIR UN MODELO DE DEEP LEARNING(CNN) DE RESTNET(RED RESIDUAL)

In [ ]:
# Se define a nivel de código, el bloque de capa red residual (RESTNET)
# Lo usaremos luego en a la arquitectura principal del modelo

def res_block_v2(X, filter, stage):

  # Bloque Convolucional
  X_copy = X

  f1 , f2, f3 = filter

  # Camino Principal
  X = layers.Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.MaxPool2D((2,2))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
  X = layers.Activation('relu')(X)

  X = layers.Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
  X = layers.Activation('relu')(X)

  X = layers.Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)


  # Camino Corto
  X_copy = layers.Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
  X_copy = layers.MaxPool2D((2,2))(X_copy)
  X_copy = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

  # Añadir
  X = layers.Add()([X,X_copy])
  X = layers.Activation('relu')(X)

  # Bloque de Identidad 1
  X_copy = X


  # Camino Principal
  X = layers.Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
  X = layers.Activation('relu')(X)

  X = layers.Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
  X = layers.Activation('relu')(X)

  X = layers.Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

  # Añadir
  X = layers.Add()([X,X_copy])
  X = layers.Activation('relu')(X)

  # Bloque de Identidad 2
  X_copy = X


  # Camino Principal
  X = layers.Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
  X = layers.Activation('relu')(X)

  X = layers.Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
  X = layers.Activation('relu')(X)

  X = layers.Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = layers.BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

  # Añadir
  X = layers.Add()([X,X_copy])
  X = layers.Activation('relu')(X)

  return X

In [ ]:
# Arquietectura principal para CNN con RestNet

# Debo usar pixeles mas grandes como 224 x 224
input_shape_v2 = (224, 224, 3)

# Tamaño del tensor de entrada
X_input2 = keras.Input(input_shape_v2)

# Zero-padding
X = layers.ZeroPadding2D((3, 3))(X_input2)

# 1 - Fase
X = layers.Conv2D(64, (7, 7), strides= (2, 2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = layers.BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = layers.Activation('relu')(X)
X = layers.MaxPooling2D((3, 3), strides= (2, 2))(X)

# 2 - Fase
X = res_block_v2(X, filter= [64, 64, 256], stage= 2)

# 3 - Fase
X = res_block_v2(X, filter= [128, 128, 512], stage= 3)

# 4 - Fase
X = res_block_v2(X, filter= [256, 256, 1024], stage= 4)

# Average Pooling
X = layers.AveragePooling2D((4, 4), name = 'Averagea_Pooling')(X)

# Capa Final
X = layers.Flatten()(X)
X = layers.Dense(2, activation = 'softmax', name = 'Dense_final', kernel_initializer= glorot_uniform(seed=0))(X)


model_clasification_leaves = keras.Model( inputs= X_input2, outputs = X, name = 'Resnet26')

model_clasification_leaves.summary()

In [ ]:
from time import gmtime, strftime
# Usamos la parada temprana para salir del entenamiento si el error de validación
# no decrece después de cierto número de epochs (paciencia)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)
fileTime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print(fileTime)
# Guardamos el mejor modelo con menor error de validación
checkpointer = ModelCheckpoint(
    filepath = "/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/Leaves_Clasification_weights "+fileTime+".hdf5",
    verbose = 1,
    save_best_only=True,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max')

In [ ]:
print("Conjunto de Entrenamiento")
print(X_train.shape, y_train.shape)
model_clasification_leaves.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

batch_size = 32

valid_generator = valid_datagen.flow(
    X_val, y_val,
    batch_size=batch_size
)
train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=batch_size
)

test_generator = test_datagen.flow(
    X_Test, y_Test,
    batch_size=batch_size
)

In [ ]:
# Visualizar algunas imágenes generadas

# # Mostrar las primeras 40 imágenes SANAS
fig = plt.figure(figsize=(15, 15))

for images, labels in train_generator:
    for i in range(40):
        ax = fig.add_subplot(5, 8, i + 1)
        ax.imshow(images[i])
        ax.axis("off")


    plt.show()
    break

In [ ]:
# TEST
fig = plt.figure(figsize=(15, 15))

for images, labels in test_generator:
    for i in range(100):
        ax = fig.add_subplot(10, 10, i + 1)
        ax.imshow(images[i])
        ax.axis("off")


    plt.show()
    break

In [ ]:
# Cargando Mejor Modelo Existente

model_clasification_leaves = load_model("/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/leaf_model Accuracy 0.978935718536377 --2023-07-09 15:04:06.h5")

score = model_clasification_leaves.evaluate(X_Test, y_Test)
print('Accuracy en la fase de Test: {}'.format(score[1]))

In [ ]:
with tf.device('/gpu:0'):
	history = model_clasification_leaves.fit(X_train, y_train, batch_size=batch_size,
		 validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // batch_size,
		epochs= 100, callbacks=[checkpointer, earlystopping])


In [ ]:
modeloCNN2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(2, activation = 'softmax', name = 'Dense_final', kernel_initializer= glorot_uniform(seed=0))
])

In [ ]:
modeloCNN2.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [ ]:
# modeloCNN2.fit(X_train, y_train, batch_size=32,
#                 validation_data=(X_val, y_val),
#                 epochs=100,
#                 steps_per_epoch=len(X_train) // 32,
#                 callbacks=[checkpointer, earlystopping])
with tf.device('/gpu:0'):
  modeloCNN2.fit(
      train_generator,
      epochs=100,
      validation_data=valid_generator,
      callbacks=[checkpointer, earlystopping]
  )

In [ ]:
fileTime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print(fileTime)
modeloCNN2.save('/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/leaf_model '+fileTime+'.h5')

In [ ]:
predictions = model_clasification_leaves.predict(X_Test)

In [ ]:
X_Test_nuevo  = X_Test.copy()
X_Test_nuevo  = X_Test_nuevo/255

In [ ]:
score = model_clasification_leaves.evaluate(X_Test, y_Test)
score_acurracy = score[1]
print('Accuracy en la fase de Test: {}'.format(score_acurracy))

In [ ]:
fileTime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print(fileTime)
model_clasification_leaves.save('/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/leaf_model Accuracy '+str(score_acurracy)+ " --"+fileTime+'.h5')

# Paso #16: EVALUAR LA EFICACIA DEL MODELO CLASIFICACIÓN

# Paso #16: EVALUAR EL MODELO DE CLASIFICACIÓN (MATRIZ DE CONFUSIÓN, ACIERTO, PRECISIÓN Y RECUPERACIÓN)

In [ ]:
model_leaf = load_model('leaf_model.h5')

In [ ]:
score = model_leaf.evaluate(X_Test, y_Test)
print('Accuracy en la fase de Test: {}'.format(score[1]))

In [ ]:
history.history.keys()

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Accuracy en el Entrenamiento')
plt.plot(epochs, val_accuracy, 'b', label='Accuracy en la Validación')
plt.title('ACCURACY')
plt.legend()

In [ ]:
# predicted_classes = model.predict_classes(X_test)
predicted_classes = np.argmax(model_clasification_leaves.predict(X_Test), axis=-1)
y_true = np.argmax(y_Test, axis=-1)

In [ ]:
label_to_text = {1:'Enferma', 0:'Sana'}

L = 5
W = 5

fig, axes = plt.subplots(L, W, figsize = (24, 24))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_Test[i])
    axes[i].set_title('Predicción = {}\n Verdadera = {}'.format(label_to_text[predicted_classes[i]], label_to_text[y_true[i]]))
    axes[i].axis('off')

plt.subplots_adjust(wspace = 1)

In [ ]:
image_hoja_sana_internet = cv2.imread('/content/copia-hoja-sana.jpg')
image_hoja_sana_internet = cv2.cvtColor(image_hoja_sana_internet, cv2.COLOR_BGR2RGB)
image_hoja_sana_internet = resize(image_hoja_sana_internet)
plt.imshow(image_hoja_sana_internet)
plt.axis('off')  # Opcional: ocultar los ejes x e y
plt.show()
image_hoja_sana_internet = image_hoja_sana_internet.reshape(1, 224, 224, 3)
predicted_sana_internet = np.argmax(model_clasification_leaves.predict(image_hoja_sana_internet), axis=-1)
label_to_text =  {1:'Enferma', 0:'Sana'}
text_prediction = label_to_text[predicted_sana_internet[0]]
print("Prediccion es: ")
print(text_prediction)



In [ ]:
model_clasification_leaves = load_model("/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/leaf_model Accuracy 0.978935718536377 --2023-07-09 15:04:06.h5")


In [ ]:
import urllib.request

# URL de la imagen
# url_sana = 'https://img.freepik.com/fotos-premium/mango-hoja-verde-aislado-sobre-fondo-blanco_42033-484.jpg'
# url_enferma = 'https://agrotrapiche.com/wp-content/uploads/2019/11/4-0-1-1024x1024.png'
# url_enferma02 = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQFcEN2V8TODwSjglUdoz8r6NhpKW2o0NE9MMg27Tid684CdeMBhHkgCXEW2lAG3SsL9rE&usqp=CAU'
url_enferma03 = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpR_Ax-iye-oPsjwa84_0spB05cj7iPE3iSp-qKis6DOmQe-QIrhSjK-4whLEeE4pyWUM&usqp=CAU'

# Descargar la imagen desde la URL
resp = urllib.request.urlopen(url_enferma03)
image_bytes = np.array(bytearray(resp.read()), dtype=np.uint8)
# Decodificar la imagen utilizando OpenCV
image_hoja_sana_internet = cv2.imdecode(image_bytes, cv2.IMREAD_COLOR)
image_hoja_sana_internet = cv2.cvtColor(image_hoja_sana_internet, cv2.COLOR_BGR2RGB)
if image is not None:
    # La imagen se ha leído correctamente
    image_hoja_sana_internet = resize(image_hoja_sana_internet)
    plt.imshow(image_hoja_sana_internet)
    plt.axis('off')  # Opcional: ocultar los ejes x e y
    plt.show()
    image_hoja_sana_internet = image_hoja_sana_internet.reshape(1, 224, 224, 3)
    predicted_sana_internet = np.argmax(model_clasification_leaves.predict(image_hoja_sana_internet), axis=-1)
    label_to_text =  {1:'Enferma', 0:'Sana'}
    text_prediction = label_to_text[predicted_sana_internet[0]]
    print("Predicción es: ")
    print(text_prediction)


In [ ]:
sanas_internet = [
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTURlJJCsTK-Owkiji6cwz3STl6iVlMTWSZNdl9NHuL_cvNZdClNTUw1LAzC_meduC19IA&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0WM435LT6Fv-YEfUov-Q78GpAv_911PgZBaU2aEcx_UMs_idUot9B1sIzz78weUKb7UM&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2VKgCbYcK6UndVyCPBIaD2pgf10SQXtilgQjjnYFSzzuC0SNUFSd8JdFi9Se7jiHeZ2M&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_xooBL5Lf-TAIwnf1ldsy_DXpg8vlFUeX44YHa3FxeMRuPXOq8salteB8H6WQdnT7koU&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSnjVPiVbVNfkdDbpR0qsDjnHL9pBeiWyrLntQ2kKP4lviT1fA7yLu2uc0JF4W7y411vzU&usqp=CAU",
  "https://us.123rf.com/450wm/yaryhee/yaryhee1606/yaryhee160600015/59215477-la-hoja-de-mango.jpg",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTqY6zWC_ejKmmMB4y0Nj5hLKkMk0X9lP0fjMPUSZoAFJ-DSqX_fNUlzsxW2bPlDlrDnQE&usqp=CAU",
  "https://us.123rf.com/450wm/aoo3771/aoo37711402/aoo3771140200369/26098871-mango-deja-aislado-en-blanco-imagen-de-fondo-en.jpg",
  "https://cdn.xxl.thumbs.canstockphoto.es/cierra-la-hoja-de-mango-sobre-fondo-blanco-almacen-de-im%C3%A1genes_csp58784959.jpg"
]

for imagen_url in sanas_internet:
  # Descargar la imagen desde la URL
  resp = urllib.request.urlopen(imagen_url)
  image_bytes = np.array(bytearray(resp.read()), dtype=np.uint8)
  # Decodificar la imagen utilizando OpenCV
  image_hoja_sana_internet = cv2.imdecode(image_bytes, cv2.IMREAD_COLOR)
  image_hoja_sana_internet = cv2.cvtColor(image_hoja_sana_internet, cv2.COLOR_BGR2RGB)
  if image is not None:
      # La imagen se ha leído correctamente
      image_hoja_sana_internet = resize(image_hoja_sana_internet)
      plt.imshow(image_hoja_sana_internet)
      plt.axis('off')  # Opcional: ocultar los ejes x e y
      plt.show()
      image_hoja_sana_internet = image_hoja_sana_internet.reshape(1, 224, 224, 3)
      predicted_sana_internet = np.argmax(model_clasification_leaves.predict(image_hoja_sana_internet), axis=-1)
      label_to_text =  {1:'Enferma', 0:'Sana'}
      text_prediction = label_to_text[predicted_sana_internet[0]]
      print("Predicción es: ")
      print(text_prediction)


In [ ]:
enfermas_internet = [
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcReuBlyPTc6q_7YCkYpqRS8bRDsLWGZcO0PPxtraUc6G1OS8fit3TPXPT6cO-D0dHDuxhg&usqp=CAU",
  "https://content.peat-cloud.com/w400/bacterial-black-spot-of-mango-mango-1552661351.jpg",
  "https://www.shutterstock.com/image-photo/alternaria-leaf-spot-on-mango-260nw-1691257909.jpg",
  "https://www.shutterstock.com/shutterstock/photos/1946321845/display_1500/stock-photo-anthracnose-disease-on-mango-leaf-caused-by-colletotrichum-gloeosporioides-on-white-background-1946321845.jpg",
  "https://live.staticflickr.com/874/42387298952_23197af4c0_b.jpg",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnjXb_R8n1MjHUI1MIGrOfv0zo4YVh5Vv6tzzWAFb-EuYyMXaJOKHFCPW_Y0wp1ME3AyI&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjdTqCNdekTir1C7jpi5CyO-n_wRLOeYSRO96mdaVEiUz99g7GftMoTp1DEUy7BE_Da0M&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxeqWWq2L_12_C1ZuUYS7-MZz6sASANFHX4V5F-EMLC3FYr_pArPCGncByfrwnJf_rNVc&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRm5JTzwfAdyn-hoQAnDk0PGEpinNsQvkvGTfcACZz56RqCjOBvxDcPciPO5m-UjdJLZwM&usqp=CAU",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSpP3r15kW_8a4n1UuKMVJhK9H-CzuKNP_FC7Te4Iz2T0d5JsfyLES0DWM3OlVlUxE9Iz4&usqp=CAU",
  "https://content.peat-cloud.com/thumbnails/phoma-blight-mango-1550581511.jpg",
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSE7Hvj9EhsxOhtxW9u8RvOIniZHyMqOvWQdw&usqp=CAU",
  "https://www.shutterstock.com/image-photo/plant-disease-mango-leaves-anthracnose-260nw-729865960.jpg",
  "https://www.shutterstock.com/image-photo/fungus-on-mango-leaves-anthracnose-260nw-1851072517.jpg",

]

for imagen_url in enfermas_internet:
  # Descargar la imagen desde la URL
  resp = urllib.request.urlopen(imagen_url)
  image_bytes = np.array(bytearray(resp.read()), dtype=np.uint8)
  # Decodificar la imagen utilizando OpenCV
  image_hoja_sana_internet = cv2.imdecode(image_bytes, cv2.IMREAD_COLOR)
  image_hoja_sana_internet = cv2.cvtColor(image_hoja_sana_internet, cv2.COLOR_BGR2RGB)
  if image is not None:
      # La imagen se ha leído correctamente
      image_hoja_sana_internet = resize(image_hoja_sana_internet)
      plt.imshow(image_hoja_sana_internet)
      plt.axis('off')  # Opcional: ocultar los ejes x e y
      plt.show()
      image_hoja_sana_internet = image_hoja_sana_internet.reshape(1, 224, 224, 3)
      predicted_sana_internet = np.argmax(model_clasification_leaves.predict(image_hoja_sana_internet), axis=-1)
      label_to_text =  {1:'Enferma', 0:'Sana'}
      text_prediction = label_to_text[predicted_sana_internet[0]]
      print("Predicción es: ")
      print(text_prediction)


In [ ]:
# Probando otro modelo
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

input_shape = (224, 224, 3)
num_classes = 2
batch_size = 32
epochs = 10

base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
# Bloquer la partie convolutionnelle du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Ajouter les nouvelles couches pour la classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Construction du modèle final
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=100, validation_data=valid_generator )

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
score = model.evaluate(X_Test, y_Test)
print('Accuracy en la fase de Test: {}'.format(score[1]))